In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
from tqdm import tqdm_notebook as tqdm
import sys
sys.path.append("../src/")
from logger import setup_logger, LOGGER
from trainer import train_lgbm
from util_tool import reduce_mem_usage
%matplotlib inline
from gensim.models import Word2Vec
pd.set_option('display.max_columns', 300)

In [2]:
# ==================
# Constant
# ==================
TRAIN_PATH = "../input/train.csv"
TEST_PATH = "../input/test.csv"
USER_PATH = "../input/user_x_anime.csv"

In [3]:
# =====================
# Settings
# =====================
SAVE_PATH = "../output/fe/fe011.feather"

In [4]:
train = pd.read_csv(TRAIN_PATH)
test = pd.read_csv(TEST_PATH)
user = pd.read_csv(USER_PATH)

In [5]:
user

,user_id,anime_id,rating,watching_status,watched_episodes
0,310366,11009,NaN,6,0
1,321597,4063,NaN,2,12
2,37696,114,9.0,2,26
3,316142,863,0.0,6,0
4,333376,32379,4.0,4,1
...,...,...,...,...,...
26992848,112800,617,NaN,2,1
26992849,34131,16498,NaN,2,25
26992850,110747,3927,8.0,2,25
26992851,94883,37497,NaN,4,6


In [6]:
def join(df):
    x = [str(e) for e in list(df)]
    return " ".join(x)
docs = user.groupby("user_id")["anime_id"].apply(join).reset_index()

In [8]:
docs

,user_id,anime_id
0,0,419 578 415 1689 433 1535 269 6098 2913 3418 2...
1,1,20 25777 37349 21881 35788 38691 38826 6547 40...
2,2,33231 35760 1389 24703 5762 5209 7875 14659 22...
3,3,306 31442 7593 13601 6880 33206 31764 19363 34...
4,4,578 1482 273 127 3470 7103 372 4725 105 819 15...
...,...,...
320135,353400,18679 18153 37349 43 10087 38691 40456 31240 3...
320136,353401,33255 33731 934 2216 10087 19 2356 10418 38000...
320137,353402,762 1142 121 1482 834 16 27 688 479 1579 68 1535
320138,353403,40221 36456 780 2966 10928 16397 31964 20507 1...


In [9]:
user_id_list = [i.split(" ") for i in docs["anime_id"]]

In [10]:
model = Word2Vec(user_id_list, vector_size=25, window=8, min_count=10, seed = 1,workers=6)

In [11]:
cols = ["MAL_ID"]
train = pd.concat([train[cols],test[cols]]).reset_index(drop=True)

In [12]:
train_embedding = np.zeros((len(train),model.vector_size))
vector_size = model.vector_size
for m,i in enumerate(tqdm(train["MAL_ID"])):
    embedding_ = np.zeros(vector_size)
    try:
        train_embedding[m,:] = model.wv[str(i)]
    except KeyError:
        pass

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  This is separate from the ipykernel package so we can avoid doing imports until


  0%|          | 0/12421 [00:00<?, ?it/s]

In [13]:
train_embedding = pd.DataFrame(train_embedding)
train_embedding.columns = [f"anime_id_emb_{i}" for i in range(len(train_embedding.columns))]

In [15]:
train_embedding = reduce_mem_usage(train_embedding)
train_embedding.to_feather(SAVE_PATH)

Memory usage of dataframe is 2.37 MB
column =  25
0
Memory usage after optimization is: 1.18 MB
Decreased by 50.0%


In [33]:
user

,user_id,anime_id,rating,watching_status,watched_episodes
0,310366,11009,NaN,6,0
1,321597,4063,NaN,2,12
2,37696,114,9.0,2,26
3,316142,863,0.0,6,0
4,333376,32379,4.0,4,1
...,...,...,...,...,...
26992848,112800,617,NaN,2,1
26992849,34131,16498,NaN,2,25
26992850,110747,3927,8.0,2,25
26992851,94883,37497,NaN,4,6
